In [1]:
import numpy as np
import fidimag
from fidimag.micro.sim import Sim
from fidimag.common.cuboid_mesh import CuboidMesh
import fidimag.common.constant as const

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib as mpl
import matplotlib.cm as cm

In [9]:
# import mayavi
from mayavi import mlab

In [3]:
Ms = 3.84e5
mesh = CuboidMesh(nx=80, ny=80, nz=1, 
                  dx=1, dy=1, dz=1, unit_length=1e-9)
sim = Sim(mesh)

CENTRE_X = (np.max(mesh.coordinates[:, 0])
            - np.min(mesh.coordinates[:, 0])) * 0.5 + np.min(mesh.coordinates[:, 0])
CENTRE_Y = (np.max(mesh.coordinates[:, 1])
            - np.min(mesh.coordinates[:, 1])) * 0.5 + np.min(mesh.coordinates[:, 1])

DIAMETER = 80
def disk(r):
    x, y = r[0] - CENTRE_X, r[1] - CENTRE_Y
    if x ** 2. + y ** 2 < (0.5 * DIAMETER) ** 2.:
        return Ms
    else:
        return 0

sim.set_Ms(disk)


In [78]:
# sim.set_m(np.load('sims/nebm/nebm_nanodisk_FeGe_d80nm-h20nm/npys/nebm_nanodisk_FeGe_d80nm-h20nm_sk-up_fm-down_800/'))
sim.set_m(np.load('sims/skyrmion_bulk_DMI_npys/m_229.npy'))

In [79]:
r = sim.mesh.coordinates
# r_sphere = (40, 40,10)
# sphere = np.logical_and((r[:, 0] - r_sphere[0]) ** 2 + (r[:, 1] - r_sphere[1]) ** 2  +  (r[:, 2] - r_sphere[2]) ** 2 < 18 ** 2,
#                         r[:, 2] > 19
#                         )

r_sphere = (40, 40, 0.5)
sphere = np.logical_and((r[:, 0] - r_sphere[0]) ** 2 + (r[:, 1] - r_sphere[1]) ** 2  +  (r[:, 2] - r_sphere[2]) ** 2 < 30 ** 2,
                        (r[:, 0] - r_sphere[0]) ** 2 + (r[:, 1] - r_sphere[1]) ** 2  +  (r[:, 2] - r_sphere[2]) ** 2 > 0 ** 2
                         )

r_rel = np.copy(r) 
r_rel[:, 0] = r_rel[:, 0] - r_sphere[0]
r_rel[:, 1] = r_rel[:, 1] - r_sphere[1]
r_rel[:, 2] = r_rel[:, 2] - r_sphere[2]

m = sim.spin.reshape(-1, 3)

In [80]:
m_inside = m[sphere]
r_inside = r[sphere]
r_rel_inside = r_rel[sphere]
# r = np.sqrt(m_inside[:, 0] ** 2 + m_inside[:, 1] ** 2)
# theta = np.arctan2(r, m_inside[:, 2])
# psi = np.arctan2(m_inside[:, 1], m_inside[:, 0])

## Stereographic

In [81]:
for i, pos in enumerate(r): 
    distance = (pos[0] - r_sphere[0]) ** 2 + (pos[1] - r_sphere[1]) ** 2
    if (distance > 5 ** 2 and distance < 8 ** 2) and i % 3 == 0:
        sphere[i] = False
    elif (distance >= 8 ** 2) and i % 2 == 0:
        sphere[i] = False

In [82]:
m_inside = m[sphere]
r_rel_inside = r_rel[sphere]
n = 5
X, Y, Z = r_rel_inside[:, 0] / n, r_rel_inside[:, 1] / n, r_rel_inside[:, 2] / n


nh = X ** 2 + Y ** 2 > 0.5 ** 2
sh = X ** 2 + Y ** 2 <= 0.5 ** 2

xs = 2 * X / (1 + X ** 2 + Y ** 2)
ys = 2 * Y / (1 + X ** 2 + Y ** 2)


# zs[sh] = -(-1 + X[sh] ** 2 + Y[sh] ** 2) / (1 + X[sh] ** 2 + Y[sh] ** 2)
# zs[nh] = (-1 + X[nh] ** 2 + Y[nh] ** 2) / (1 + X[nh] ** 2 + Y[nh] ** 2)
zs = (-1 + X ** 2 + Y ** 2) / (1 + X ** 2 + Y ** 2)

In [83]:
fig = mlab.figure(bgcolor=(1, 1, 1), fgcolor=(0, 0, 0), size=(1000, 1000))

q = mlab.quiver3d(xs, ys, zs,
                  m_inside[:, 0], m_inside[:, 1], m_inside[:, 2],
                  scalars=m_inside[:, 2],
                  resolution=80, line_width=3, colormap='RdYlBu'
                  )
q.glyph.color_mode = 'color_by_scalar'
q.glyph.glyph_source.glyph_source = q.glyph.glyph_source.glyph_dict['cone_source']
q.glyph.glyph_source.glyph_position = 'center'
q.glyph.glyph_source.glyph_source.resolution = 100
q.glyph.glyph_source.glyph_source.radius = 0.4
q.glyph.glyph.scale_factor = 0.2

mlab.points3d([0], [0], [0], 1, scale_factor=2, resolution=50, color=(1, 1, 1))

fig.scene.x_minus_view()
# fig.scene.camera.position = [4.7252755610085764, 4.2488252455584137, 3.4293239539205569]
# fig.scene.camera.focal_point = [0.15362982920521273, 0.1706501120580434, -0.18238092178350548]
# fig.scene.camera.view_angle = 30.0
# fig.scene.camera.view_up = [-0.37799885829766694, -0.33916983382114257, 0.86144105251108094]
# fig.scene.camera.clipping_range = [3.3690203293112528, 12.014591840217518]

# mlab.savefig('skyrmion_stereographic.pov')

mlab.show()

# Povray